FRED data: 
- VIX (volatility index): Measures market fear/uncertainty. Spikes during crises.
- 10Y treasury: 
- 2Y treasury:
- Yield curve: (10Y - 2Y) Predicts economic cycles. Inversion often precedes recessions
- BAA corporate Bond: Captures perceived credit risk in corporate bonds. Widens during financial stress.
- CPI: Indicates inflationary environment. Influences policy and real returns.
- Unemployment rate: Reflects slack in the labour market. Lags the business cycle but still valuable.
- Fed Funds Rate: Measures monetary policy stance. Sharp changes can trigger regime shifts.
- GDP growth: measures real economic activity (however it is in quarterly not monthly so may have to forward fill to each corresponding month i.e. fill each missing month with the most recent known GDP value)
- Credit spread (BAA - GS10) (calculated)
- YoY Inflation = 12-month % change in CPI 

API key: 043d7bab55a44c38c28c692cbfddae0e 

In [29]:
import pandas as pd
from datetime import datetime
from fredapi import Fred
from dotenv import load_dotenv
import os


In [30]:
load_dotenv()

fred_api_key = os.getenv("FRED_API_KEY")

f = Fred(api_key=fred_api_key) # Instantiate FRED with API key 


In [31]:
# Define series of interest
series = {
    'VIX': 'VIXCLS',
    '10Y': 'GS10',
    '2Y': 'GS2',
    'YieldCurve': 'T10Y2Y',
    'BAA': 'BAA', # Corporate bond yield (credit risk)
    'CPI': 'CPIAUCSL',
    'UnemploymentRate': 'UNRATE',
    'FedFundsRate': 'FEDFUNDS',
    'GDP_YoY': 'A191RL1Q225SBEA'
}

In [32]:
start_date = '1963-07-01'
end_date = '2025-05-31'


In [33]:
macro_data = {}
for name, code in series.items():
    macro_data[name] = f.get_series(code, observation_start=start_date, observation_end=end_date)


In [34]:
macro_df = pd.DataFrame(macro_data) # Convert each to a dataframe


In [35]:
macro_df['Date'] = macro_df.index # Add date as a column


In [36]:
# Compute derived columns
macro_df['CreditSpread'] = macro_df['BAA'] - macro_df['10Y']
macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100
macro_df['GDP_YoY'] = macro_df['GDP_YoY'].ffill()


C:\Users\User\AppData\Local\Temp\ipykernel_21872\1166655369.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100


In [37]:
# Reset index and reorder
macro_df.reset_index(drop=True, inplace=True)
macro_df = macro_df[['Date', 'VIX', '2Y', '10Y', 'YieldCurve', 'CreditSpread',
                 'FedFundsRate', 'Inflation_YoY', 'UnemploymentRate', 'GDP_YoY']]

In [38]:

macro_df.head()


,Date,VIX,2Y,10Y,YieldCurve,CreditSpread,FedFundsRate,Inflation_YoY,UnemploymentRate,GDP_YoY
0,1963-07-01,NaN,NaN,4.02,NaN,0.82,3.02,NaN,5.6,9.1
1,1963-08-01,NaN,NaN,4.00,NaN,0.83,3.49,NaN,5.4,9.1
2,1963-09-01,NaN,NaN,4.08,NaN,0.76,3.48,NaN,5.5,9.1
3,1963-10-01,NaN,NaN,4.11,NaN,0.72,3.50,NaN,5.5,2.6
4,1963-11-01,NaN,NaN,4.12,NaN,0.72,3.48,NaN,5.7,2.6


In [40]:
macro_df.to_csv('macro_data.csv', index=False)
